In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.4 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requ

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [14]:
#load pdf files
loader = PyPDFDirectoryLoader("/kaggle/input/info-brochure")
data = loader.load()

In [ ]:
print(data)

In [15]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [16]:
len(text_chunks)

21

In [ ]:
#get the third chunk
text_chunks[2]

In [17]:
!pip install transformers datasets


In [18]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import warnings, logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
# #Step 06:Downlaod the Embeddings
# model_nm = 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
# model = AutoModelForSequenceClassification.from_pretrained(model_nm, return_dict=True)
# tokenizer = AutoTokenizer.from_pretrained(model_nm)


In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/kaggle/input/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /kaggle/input/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:     

In [22]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [32]:
#rainmaker
# query1 = "Who compiled this pdf?"
# query2 = "What is the institute mentioned in the pdf"
# query3 = "What do you think was the most challenging part of the students' cloud-seeding project?"
# query4 = "Summarize the Story"

In [33]:
#info brochure
query1 = "What is admission criteria for Mtech program"

query2 = "Summarize admission criteria for various PhD programs"

query3 = "What is criteria for Admission of Foreign Nationals tp PhD program?"

query4 = "What are various specializations under Electronics Engineering?"

query5 = "What are number of seats available for Electronics Mtech program?"

query6 =  '''I am highly interested in Computational Neuroscience. 
            Does any department offer this in research program?'''

query7 = "Can I get registered through Kirloskar Electric Limited, Bangalore ?"

In [23]:
#iit_bhu info brochure 
query1 = "What are selection criteria for various PhD programs?"
query2 = "What is selection criteria for Mtech Program"
query3 = "can foreign "

In [24]:
qa.run(query1)


llama_print_timings:        load time =    2049.53 ms
llama_print_timings:      sample time =     165.92 ms /   256 runs   (    0.65 ms per token,  1542.91 tokens per second)
llama_print_timings: prompt eval time =  385953.09 ms /  1660 tokens (  232.50 ms per token,     4.30 tokens per second)
llama_print_timings:        eval time =  104071.18 ms /   255 runs   (  408.12 ms per token,     2.45 tokens per second)
llama_print_timings:       total time =  490769.44 ms /  1915 tokens


"\n\nThe eligibility criteria, qualifications and minimum requirements vary for different PhD programs based on the discipline of study. For instance, Ph.D. in Humanistic Studies requires a Master's degree with minimum CPI of 6.00 or 75% marks at the bachelor's level, while Ph.D. in Architecture, Planning and Design requires a minimum of 75% marks at the bachelor's degree level or M.Arch., M.Plan., M.Des. or related discipline/area with minimum CPI of 60%. The selection criteria for interdisciplinary programs like Ph.D. in Systems Engineering, Industrial Management and Bio-chemical Engineering/Bio-medical Engineering/Materials Science and Technology require a minimum of 75% marks at the bachelor's degree level or master's degree with minimum CPI of 75%.\n\nIt is important to note that candidates applying for admission to M.Tech. Programmes in different departments/schools as well as M.Pharm. programme are required to fill the primary and additional programme/discipline/specialization i

In [34]:
qa.run(query3)

Llama.generate: 45 prefix-match hit, remaining 2149 prompt tokens to eval

llama_print_timings:        load time =    2049.53 ms
llama_print_timings:      sample time =      78.55 ms /   119 runs   (    0.66 ms per token,  1514.96 tokens per second)
llama_print_timings: prompt eval time =  497725.19 ms /  2149 tokens (  231.61 ms per token,     4.32 tokens per second)
llama_print_timings:        eval time =   48181.40 ms /   118 runs   (  408.32 ms per token,     2.45 tokens per second)
llama_print_timings:       total time =  546196.62 ms /  2267 tokens


"\n\nForeign nationals can apply for admission to PhD programs under the Full Time Regular Registration Category if they have a master's degree in an allied subject or science from a recognized institution, a minimum CPI of 6.0 on a 10-point scale (or 60% marks) in the qualifying degree. They must also have at least two years of professional experience and meet the other eligibility criteria for their chosen program. Financial assistance is available in the form of Institute Assistantships, but they must qualify for GATE/GPAT."

In [28]:
qa.run(query4)

Llama.generate: 45 prefix-match hit, remaining 1720 prompt tokens to eval

llama_print_timings:        load time =    2049.53 ms
llama_print_timings:      sample time =      21.66 ms /    34 runs   (    0.64 ms per token,  1569.42 tokens per second)
llama_print_timings: prompt eval time =  391941.49 ms /  1720 tokens (  227.87 ms per token,     4.39 tokens per second)
llama_print_timings:        eval time =   13415.74 ms /    33 runs   (  406.54 ms per token,     2.46 tokens per second)
llama_print_timings:       total time =  405452.34 ms /  1753 tokens


' Electronics Engineering offers the following specializations: Microwave Engineering, Digital Techniques and Instrumentation, Microelectronics, Communication System Engineering.'

In [29]:
qa.run(query5)

Llama.generate: 46 prefix-match hit, remaining 875 prompt tokens to eval

llama_print_timings:        load time =    2049.53 ms
llama_print_timings:      sample time =      13.67 ms /    22 runs   (    0.62 ms per token,  1609.25 tokens per second)
llama_print_timings: prompt eval time =  193658.38 ms /   875 tokens (  221.32 ms per token,     4.52 tokens per second)
llama_print_timings:        eval time =    7800.24 ms /    21 runs   (  371.44 ms per token,     2.69 tokens per second)
llama_print_timings:       total time =  201511.31 ms /   896 tokens


' The number of seats available for Electronics M.Tech. Programme is 19.'

In [30]:
qa.run(query6)

Llama.generate: 45 prefix-match hit, remaining 1569 prompt tokens to eval

llama_print_timings:        load time =    2049.53 ms
llama_print_timings:      sample time =     107.20 ms /   164 runs   (    0.65 ms per token,  1529.81 tokens per second)
llama_print_timings: prompt eval time =  354839.52 ms /  1568 tokens (  226.30 ms per token,     4.42 tokens per second)
llama_print_timings:        eval time =   64390.60 ms /   164 runs   (  392.63 ms per token,     2.55 tokens per second)
llama_print_timings:       total time =  419591.13 ms /  1732 tokens


' Yes, Computational Neuroscience is a research area that can be recognized by the Institute as per the following procedure. On the recommendation of the DPGC, the Dean (Academic Affairs) will constitute a committee to assess and approve an R & D organization/industry for admission of sponsored candidates to carry out Ph.D. research in a specified area. The committee may, upon inspection, also approve all the areas in which R & D activities are going on in that organization. An application for admission from a candidate working in the approved organization will be considered only if he/she wishes to work in the approved area. Therefore, one has to find an R&D organization or an industry that is recognized by the Institute and offers Computational Neuroscience as a research area.'